# **Рубежный контроль №2**
**Студент:**Ян Тяньци   
**Группа:** ИУ5-21М    
**Тема:** Методы обработки текстов 

**Классификатор №1:** LogisticRegression  
**Классификатор №2:** Multinomial Naive Bayes - MNB

In [7]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
data_root = '/content/drive/MyDrive/MMO'
print(os.listdir(data_root))

['Musical_instruments_reviews.csv']


In [ ]:
!unzip /content/drive/MyDrive/MMO/Musical_instruments_reviews.csv

In [12]:
data = pd.read_csv("Musical_instruments_reviews.csv")
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [26]:
data.shape

(10261, 9)

In [27]:
vocab_list = data['summary'].tolist()
vocab_list[1:10]

['Jake',
 'It Does The Job Well',
 'GOOD WINDSCREEN FOR THE MONEY',
 'No more pops when I record my vocals.',
 'The Best Cable',
 "Monster Standard 100 - 21' Instrument Cable",
 "Didn't fit my 1996 Fender Strat...",
 'Great cable',
 'Best Instrument Cables On The Market']

In [28]:
vocabVect = CountVectorizer()
vocabVect.fit(vocab_list)
corpusVocab = vocabVect.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 4219


In [29]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

jake=2016
it=2002
does=1101
the=3722
job=2033
well=4085
windscreen=4120
for=1504
money=2406


In [31]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

**Использование класса *CountVectorizer***  
Подсчитывает количество слов словаря, входящих в данный текст.

In [34]:
test_features = vocabVect.transform(data)
test_features

<9x4219 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [35]:
test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [36]:
len(test_features.todense()[0].getA1())

4219

In [59]:
[i for i in test_features.todense()[0].getA1() if i>0]

[]

In [38]:
vocabVect.get_feature_names()[100:120]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['57',
 '58',
 '5ft',
 '5mm',
 '5th',
 '60',
 '6024',
 '60g',
 '60mm',
 '60s',
 '6112',
 '65',
 '6b',
 '70',
 '700',
 '7007si',
 '700s',
 '70s',
 '72',
 '73mm']

**Использование класса *TfidfVectorizer***  
Вычисляет специфичность текста в корпусе текстов на основе метрики *TF-IDF*.

In [39]:
tfidfv = TfidfVectorizer(ngram_range=(1,3))
tfidf_ngram_features = tfidfv.fit_transform(vocab_list)
# tfidf_ngram_features = tfidfv.fit_transform(data['overview'])
tfidf_ngram_features

<10261x41044 sparse matrix of type '<class 'numpy.float64'>'
	with 98741 stored elements in Compressed Sparse Row format>

In [40]:
tfidf_ngram_features.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
len(tfidf_ngram_features.todense()[0].getA1())

41044

In [42]:
[i for i in tfidf_ngram_features.todense()[0].getA1() if i>0]

[1.0]

**Решение задачи анализа тональности текста**  
С использованием кросс-валидации попробуем применить к корпусу текстов различные варианты векторизации и классификации.

In [44]:
data2 = pd.read_csv("pulitzer-circulation-data.csv")
data2.head()

,Newspaper,"Daily Circulation, 2004","Daily Circulation, 2013","Change in Daily Circulation, 2004-2013","Pulitzer Prize Winners and Finalists, 1990-2003","Pulitzer Prize Winners and Finalists, 2004-2014","Pulitzer Prize Winners and Finalists, 1990-2014"
0,USA Today,"2,192,098","1,674,306",-24%,1,1,2
1,Wall Street Journal,"2,101,017","2,378,827",+13%,30,20,50
2,New York Times,"1,119,027","1,865,318",+67%,55,62,117
3,Los Angeles Times,"983,727","653,868",-34%,44,41,85
4,Washington Post,"760,034","474,767",-38%,52,48,100


In [49]:
def VectorizeAndClassify(vectorizers_list, classifiers_list):
    for v in vectorizers_list:
        for c in classifiers_list:
            pipeline1 = Pipeline([("vectorizer", v), ("classifier", c)])
            score = cross_val_score(pipeline1, data['summary'], data['overall'], scoring='accuracy', cv=3).mean()
            print('Векторизация - {}'.format(v))
            print('Модель для классификации - {}'.format(c))
            print('Accuracy = {}'.format(score))
            print('===========================')

In [50]:
vectorizers_list = [CountVectorizer(vocabulary = corpusVocab), TfidfVectorizer(vocabulary = corpusVocab)]
classifiers_list = [LogisticRegression(C=3.0), MultinomialNB()]
VectorizeAndClassify(vectorizers_list, classifiers_list)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '009': 2, '01': 3, '010': 4,
                            '02': 5, '042': 6, '046': 7, '0521': 8, '073': 9,
                            '07mkii': 10, '09': 11, '099': 12, '10': 13,
                            '100': 14, '100x': 15, '108': 16, '10ft': 17,
                            '10p': 18, '10s': 19, '10w': 20, '11': 21,
                            '110v': 22, '116': 23, '12': 24, '120': 25,
                            '12at7': 26, '12ax7': 27, '13': 28, '13th': 29, ...})
Модель для классификации - LogisticRegression(C=3.0)
Accuracy = 0.6790759174072765
Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '009': 2, '01': 3, '010': 4,
                            '02': 5, '042': 6, '046': 7, '0521': 8, '073': 9,
                            '07mkii': 10, '09': 11, '099': 12, '10': 13,
                            '100': 14, '100x': 15, '108': 16, '10ft': 17,
                            '10p': 18, '10s': 19, '10w': 20,

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Векторизация - TfidfVectorizer(vocabulary={'00': 0, '000': 1, '009': 2, '01': 3, '010': 4,
                            '02': 5, '042': 6, '046': 7, '0521': 8, '073': 9,
                            '07mkii': 10, '09': 11, '099': 12, '10': 13,
                            '100': 14, '100x': 15, '108': 16, '10ft': 17,
                            '10p': 18, '10s': 19, '10w': 20, '11': 21,
                            '110v': 22, '116': 23, '12': 24, '120': 25,
                            '12at7': 26, '12ax7': 27, '13': 28, '13th': 29, ...})
Модель для классификации - LogisticRegression(C=3.0)
Accuracy = 0.6929149907149569
Векторизация - TfidfVectorizer(vocabulary={'00': 0, '000': 1, '009': 2, '01': 3, '010': 4,
                            '02': 5, '042': 6, '046': 7, '0521': 8, '073': 9,
                            '07mkii': 10, '09': 11, '099': 12, '10': 13,
                            '100': 14, '100x': 15, '108': 16, '10ft': 17,
                            '10p': 18, '10s': 19, '10w': 20,

**Разделим выборку на обучающую и тестовую**

In [52]:
X_train, X_test, y_train, y_test = train_test_split(data['summary'], data['overall'], test_size=0.5, random_state=1)


In [53]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [54]:
sentiment(TfidfVectorizer(), LogisticRegression(C=5.0))


Метка 	 Accuracy
1.0 	 0.1016949152542373
2.0 	 0.09090909090909091
3.0 	 0.24031007751937986
4.0 	 0.2028301886792453
5.0 	 0.9129050925925926


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [55]:
sentiment(TfidfVectorizer(ngram_range=(1,3)), LogisticRegression(C=5.0))


Метка 	 Accuracy
1.0 	 0.1016949152542373
2.0 	 0.12727272727272726
3.0 	 0.2248062015503876
4.0 	 0.19245283018867926
5.0 	 0.9189814814814815


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [56]:
sentiment(TfidfVectorizer(ngram_range=(2,3)), LogisticRegression(C=5.0))


Метка 	 Accuracy
1.0 	 0.0423728813559322
2.0 	 0.05454545454545454
3.0 	 0.12144702842377261
4.0 	 0.13679245283018868
5.0 	 0.9354745370370371


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [57]:
sentiment(TfidfVectorizer(ngram_range=(1,4)), LogisticRegression(C=5.0))


Метка 	 Accuracy
1.0 	 0.09322033898305085
2.0 	 0.12727272727272726
3.0 	 0.2248062015503876
4.0 	 0.19245283018867926
5.0 	 0.9227430555555556


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [58]:
sentiment(TfidfVectorizer(ngram_range=(2,4)), LogisticRegression(C=5.0))


Метка 	 Accuracy
1.0 	 0.0423728813559322
2.0 	 0.05454545454545454
3.0 	 0.11627906976744186
4.0 	 0.14150943396226415
5.0 	 0.9380787037037037


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


**Вывод:**  
Как видно из результатов, лучшую точность показал *CountVectorizer* и *MultinomialNB*.  
Точность составила 68,1%.